# Clock Modeling

In [1]:
import re
import json

## Parsing LMK Roadmap

In [2]:
!head -20 lmk04828b_regmap.txt

0x000:
        RESET (end:7|start:7|width:1)
        0 (end:6|start:6|width:1)
        0 (end:5|start:5|width:1)
        SPI_3WIRE_DIS (end:4|start:4|width:1)
        0 (end:3|start:3|width:1)
        0 (end:2|start:2|width:1)
        0 (end:1|start:1|width:1)
        0 (end:0|start:0|width:1)

0x002:
        0 (end:7|start:7|width:1)
        0 (end:6|start:6|width:1)
        0 (end:5|start:5|width:1)
        0 (end:4|start:4|width:1)
        0 (end:3|start:3|width:1)
        0 (end:2|start:2|width:1)
        0 (end:1|start:1|width:1)
        POWERDOWN (end:0|start:0|width:1)



In [3]:
with open("lmk04828b_regmap.txt", "r") as f:
    regmap_lines = f.read().strip().split("\n")

In [65]:
class Field:
    def __init__(self, parent, end, start, name):
        self.parent = parent
        self.end = end
        self.start = start
        self.name = name
        self.value = 0
        
        self.width = end-start+1
        self.mask = ((1 << width)-1) << start
        
    @property
    def __dict__(self):
        return {"fieldtype": "normal", "end": self.end, "start": self.start, "name": self.name, "description": "", "default": 0, "valid": {
                    "type": "enum",
                    "values": [
                        { "value": 0, "name": "NAME", "description": "Descr" }
                    ]
               }}
    
    def get(self):
        return (self.value << self.start) & self.mask
    
class ConstantField:
    def __init__(self, parent, end, start, val):
        self.parent = parent
        self.end = end
        self.start = start
        self.val = val
        
    @property
    def __dict__(self):
        return {"fieldtype": "constant", "end": self.end, "start": self.start, "value": self.val}
    
    def get(self):
        return self.val << self.start

class Register:
    def __init__(self, addr):
        self.addr = addr
        self.fields = []
        
    @property
    def __dict__(self):
        return {"addr": self.addr, "fields": self.fields}
    
    def add_field(self, end, start, name):
        if name == "0" or name == "1":
            self.fields.append(ConstantField(self, end, start, int(name)))
        elif isinstance(name, int):
            self.fields.append(ConstantField(self, 15, 0, name))
        else:
            field = Field(self, end, start, name)
            self.fields.append(field)
            return field

In [5]:
reg_pattern = re.compile(r"(0|[A-Z0-9_\[\]:a-z]+) \(end:(\d)\|start:(\d)\|width:(\d)\)")

In [6]:
regmap = {}
regmap_fields = {}
registers = []

for line in regmap_lines:
    if line.endswith(":"):
        addr = int(line[:-1], 16)
        
        reg = Register(addr)
        regmap[addr] = reg
        registers.append(reg)
        
    elif len(line.strip()) > 0:
        match = reg_pattern.match(line.strip())
        if match is None:
            raise RuntimeError("Unknown line detected: " + line.strip())
        
        regname = match.group(1)
        if regname == "X":
            continue

        end = int(match.group(2))
        start = int(match.group(3))
        width = int(match.group(4))
        
        # print(f"regmap[{hex(addr):>7}] [{end}:{start}]: {regname:20}")
        field = reg.add_field(end, start, regname)
        
        if field is not None:
            assert regname not in regmap_fields
            regmap_fields[regname] = field

In [7]:
class RegisterSerializer(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Register) or isinstance(obj, Field) or isinstance(obj, ConstantField):
            return obj.__dict__
        else:
            return json.JSONEncoder.default(self, obj)

In [8]:
with open("lmk04828b_regmap_fix.json", "w") as f:
    json.dump(registers, f, cls=RegisterSerializer, indent=4)

In [9]:
def get_clkout_odl_idl_div(addr, p):
    dclkout_defaults = {0: 2,
                        2: 4,
                        4: 8,
                        6: 8,
                        8: 8,
                        10: 8,
                        12: 2}
    return {
        "addr": addr,
        "fields": [
            {
                "fieldtype": "constant",
                "end": 7,
                "start": 7,
                "value": 0
            },
            {
                "fieldtype": "normal",
                "end": 6,
                "start": 6,
                "name": f"CLKout{p[0]}_{p[1]}_ODL",
                "description": f"Output drive level. Setting this bit increases the current to the CLKout{p[0]}_{p[1]} output buffers, which can slightly improve noise floor.",
                "default": 0,
                "valid": {
                    "type": "int"
                }
            },
            {
                "fieldtype": "normal",
                "end": 5,
                "start": 5,
                "name": f"CLKout{p[0]}_{p[1]}_IDL",
                "description": f"Input drive level. Setting this bit increases the current to the clock distribution buffer sourcing CLKout{p[0]}_{p[1]}, which can slightly improve noise floor.",
                "default": 0,
                "valid": {
                    "type": "int"
                }
            },
            {
                "fieldtype": "normal",
                "end": 4,
                "start": 0,
                "name": f"DCLKout{p[0]}_DIV",
                "description": f"DCLKout{p[0]}_DIV sets the divide value for the clock output; the divide may be even or odd. Both even or odd divides output a 50% duty cycle clock if duty cycle correction (DCC) is selected. Divider is unused if DCLKout{p[0]}_MUX = 2 (bypass), equivalent divide of 1.",
                "default": dclkout_defaults[p[0]],
                "valid": {
                    "type": "int"
                }
            }
        ]
    }

def get_clkout_ddly(addr, p):
    return {
        "addr": addr,
        "fields": [
            {
                "fieldtype": "normal",
                "end": 7,
                "start": 4,
                "name": f"DCLKout{p}_DDLY_CNTH",
                "description": "Number of clock cycles the output is high when digital delay is engaged.",
                "default": 5,
                "valid": {
                    "type": "int"
                }
            },
            {
                "fieldtype": "normal",
                "end": 3,
                "start": 0,
                "name": f"DCLKout{p}_DDLY_CNTL",
                "description": "Number of clock cycles the output is low when dynamic digital delay is engaged.",
                "default": 5,
                "valid": {
                    "type": "int"
                }
            }
        ]
    }

def get_clkout_ddlyd(addr, p):
    return {
        "addr": addr,
        "fields": [
            {
                "fieldtype": "normal",
                "end": 7,
                "start": 4,
                "name": f"DCLKout{p}_DDLYd_CNTH",
                "description": "Range 2 to 16.  Upon trigger of dynamic digital delay, the output will be low for DEVCLKdddlyLOW 'clock distribution path' cycles and then a high for DEVCLKdddlyHIGH 'clock distribution path' cycles.",
                "default": 5,
                "valid": {
                    "type": "int"
                }
            },
            {
                "fieldtype": "normal",
                "end": 3,
                "start": 0,
                "name": f"DCLKout{p}_DDLYd_CNTL",
                "description": "Range 2 to 16.  Upon trigger of dynamic digital delay, the output will be low for DEVCLKdddlyLOW 'clock distribution path' cycles and then a high for DEVCLKdddlyHIGH 'clock distribution path' cycles.",
                "default": 5,
                "valid": {
                    "type": "int"
                }
            }
        ]
    }

def get_clkout_adly(addr, i):
    return {
        "addr": addr,
        "fields": [
            {
                "fieldtype": "normal",
                "end": 7,
                "start": 3,
                "name": f"DCLKout{i}_ADLY",
                "description": f"Device clock analog delay value. Delay step size is 25 ps. DCLKout{i}_ADLY_PD = 0 (DCLK analog delay powered up) also adds a fixed 500-ps delay. Effective range is 500 ps to 1075 ps.",
                "default": 0,
                "valid": {
                    "type": "int"
                }
            },
            {
                "fieldtype": "normal",
                "end": 2,
                "start": 2,
                "name": f"DCLKout{i}_ADLY_MUX",
                "description": f"This register selects the input to the analog delay for the device clock. Used when DCLKout{i}_MUX = 3.",
                "default": 0,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "NO_DUTY_CYCLE_CORRECTION",
                            "description": f"Divided without duty cycle correction or half step. DCLKout{i}_DIV = 1 is not valid when DCLKout{i}_MUX = 0. DCLKout{i}_DIV = 1 is valid for DCLKout{i}_MUX = 1, or DCLKout{i}_MUX = 3 and DCLKout{i}_ADLY_MUX = 1."
                        },
                        {
                            "value": 1,
                            "name": "WITH_DUTY_CYCLE_CORRECTION",
                            "description": "Divided with duty cycle correction and half step."
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 1,
                "start": 0,
                "name": f"DCLKout{i}_MUX",
                "description": "This selects the input to the device clock buffer.",
                "default": 0,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "DIVIDER_ONLY",
                            "description": f"Divider Only. DCLKout{i}_DIV = 1 is not valid when DCLKout{i}_MUX = 0. DCLKout{i}_DIV = 1 is valid for DCLKout{i}_MUX = 1, or DCLKout{i}_MUX = 3 and DCLKout{i}_ADLY_MUX = 1."
                        },
                        {
                            "value": 1,
                            "name": "DIVIDER_WITH_DUTY_CYCLE_CORRECTION",
                            "description": "Divider with duty cycle Correction and half step"
                        },
                        {
                            "value": 2,
                            "name": "BYPASS",
                            "description": "Bypass"
                        },
                        {
                            "value": 3,
                            "name": "ANALOG_DELAY_AND_DIVIDER",
                            "description": "Analog delay + divider"
                        }
                    ]
                }
            }
        ]
    }

def get_clkout_hs_mux(addr, p):
    return {
        "addr": addr,
        "fields": [
            {
                "fieldtype": "constant",
                "end": 7,
                "start": 7,
                "value": 0
            },
            {
                "fieldtype": "normal",
                "end": 6,
                "start": 6,
                "name": f"DCLKout{p}_HS",
                "description": "Sets the device clock half step value. Half step must be zero (0) for a divide of 1.",
                "default": 0,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "ZERO_CYCLES",
                            "description": "0 cycles"
                        },
                        {
                            "value": 1,
                            "name": "MINUS_HALF_CYCLE",
                            "description": "-0.5 cycles"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 5,
                "start": 5,
                "name": f"SDCLKout{p+1}_MUX",
                "description": f"Sets the input the the SDCLKout{p} outputs.",
                "default": 0,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "DEVICE_CLOCK_OUTPUT",
                            "description": "Device clock output"
                        },
                        {
                            "value": 1,
                            "name": "SYSREF_OUTPUT",
                            "description": "SYSREF output"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 4,
                "start": 1,
                "name": f"SDCLKout{p+1}_DDLY",
                "description": "Sets the number of VCO cycles to delay the SDCLKout by",
                "default": 0,
                "valid": {
                    "type": "int"
                }
            },
            {
                "fieldtype": "normal",
                "end": 0,
                "start": 0,
                "name": f"SDCLKout{p+1}_HS",
                "description": "Sets the SYSREF clock half-step value.",
                "default": 0,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "ZERO_CYCLES",
                            "description": "0 cycles"
                        },
                        {
                            "value": 1,
                            "name": "MINUS_HALF_CYCLE",
                            "description": "-0.5 cycles"
                        }
                    ]
                }
            }
        ]
    }

def get_sdclk_adly_en(addr, p):
    return {
        "addr": addr,
        "fields": [
            {
                "fieldtype": "constant",
                "end": 7,
                "start": 7,
                "value": 0
            },
            {
                "fieldtype": "constant",
                "end": 6,
                "start": 6,
                "value": 0
            },
            {
                "fieldtype": "constant",
                "end": 5,
                "start": 5,
                "value": 0
            },
            {
                "fieldtype": "normal",
                "end": 4,
                "start": 4,
                "name": f"SDCLKout{p+1}_ADLY_EN",
                "description": "Enables analog delay for the SYSREF output",
                "default": 0,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "DISABLED",
                            "description": "Disabled"
                        },
                        {
                            "value": 1,
                            "name": "ENABLED",
                            "description": "Enabled"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 3,
                "start": 0,
                "name": f"SDCLKout{p+1}_ADLY",
                "description": f"Sets the analog delay value for the SYSREF output. Step size is 150 ps, except first step (600 ps). SDCLKout{p+1}_ADLY_EN = 1 (SDCLK analog delay enabled) also adds a fixed 700-ps delay. Effective range is 700 ps to 2950 ps.",
                "default": 0,
                "valid": {
                    "type": "int"
                }
            }
        ]
    }

def get_dclkout_ddly_pd(addr, p):
    powerdown_defaults = {0: 1,
                          2: 1,
                          4: 0,
                          6: 0,
                          8: 0,
                          10: 0,
                          12: 1}
    return {
        "addr": addr,
        "fields": [
            {
                "fieldtype": "normal",
                "end": 7,
                "start": 7,
                "name": f"DCLKout{p}_DDLY_PD",
                "description": "Powerdown the device clock digital delay circuitry.",
                "default": 0,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "ENABLED",
                            "description": "Enabled"
                        },
                        {
                            "value": 1,
                            "name": "POWERDOWN",
                            "description": "Powerdown"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 6,
                "start": 6,
                "name": f"DCLKout{p}_HSg_PD",
                "description": "Powerdown the device clock glitchless half-step feature.",
                "default": 1,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "ENABLED",
                            "description": "Enabled"
                        },
                        {
                            "value": 1,
                            "name": "POWERDOWN",
                            "description": "Powerdown"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 5,
                "start": 5,
                "name": f"DCLKout{p}_ADLYg_PD",
                "description": "Powerdown the device clock glitchless analog delay feature.",
                "default": 1,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "ENABLED",
                            "description": "Enabled"
                        },
                        {
                            "value": 1,
                            "name": "POWERDOWN",
                            "description": "Powerdown"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 4,
                "start": 4,
                "name": f"DCLKout{p}_ADLY_PD",
                "description": "Powerdown the device clock analog delay feature.",
                "default": 1,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "ENABLED",
                            "description": "Enabled"
                        },
                        {
                            "value": 1,
                            "name": "POWERDOWN",
                            "description": "Powerdown"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 3,
                "start": 3,
                "name": f"CLKout{p}_{p+1}_PD",
                "description": f"Powerdown the clock group defined by {p} and {p+1}.",
                "default": powerdown_defaults[p],
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "ENABLED",
                            "description": "Enabled"
                        },
                        {
                            "value": 1,
                            "name": "POWERDOWN",
                            "description": "Powerdown"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 2,
                "start": 1,
                "name": f"SDCLKout{p+1}_DIS_MODE",
                "description": "Configures the output state of the SYSREF when disabled",
                "default": 0,
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "ACTIVE",
                            "description": "Active in normal operation"
                        },
                        {
                            "value": 1,
                            "name": "GLOBAL_PD_LOGIC_LOW",
                            "description": "If SYSREF_GBL_PD = 1, the output is a logic low, otherwise it is active."
                        },
                        {
                            "value": 2,
                            "name": "GLOBAL_PD_LOGIC_HIGH",
                            "description": "If SYSREF_GBL_PD = 1, the output is a nominal Vcm voltage, otherwise it is active. If LVPECL mode is used with emitter resistors to ground, the output Vcm is ~0 V, and each pin is ~0 V."
                        },
                        {
                            "value": 3,
                            "name": "OUTPUT_NOMINAL_VCM_VOLTAGE",
                            "description": "Output is a nominal Vcm voltage. If LVPECL mode is used with emitter resistors to ground, the output Vcm is ~0 V, and each pin is ~0 V."
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 0,
                "start": 0,
                "name": f"SDCLKout{p+1}_PD",
                "description": f"Powerdown SDCLKout{p} and set to the state defined by SDCLKout{p}_DIS_MODE",
                "default": 0,
                "valid": {
                    "type": "enum",
                    "default": 1,
                    "values": [
                        {
                            "value": 0,
                            "name": "ENABLED",
                            "description": "Enabled"
                        },
                        {
                            "value": 1,
                            "name": "POWERDOWN",
                            "description": "Powerdown"
                        }
                    ]
                }
            }
        ]
    }

def get_sdclkout_pol_fmt(addr, p):
    fmt_defaults = {0: 0,
                    2: 0,
                    4: 1,
                    6: 1,
                    8: 1,
                    10: 1,
                    12: 0}
    
    return {
        "addr": addr,
        "fields": [
            {
                "fieldtype": "normal",
                "end": 7,
                "start": 7,
                "name": f"SDCLKout{p+1}_POL",
                "description": f"Sets the polarity of clock on SDCLKout{p+1} when device clock output is selected with SDCLKout{p+1}_MUX.",
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "NORMAL",
                            "description": "Normal"
                        },
                        {
                            "value": 1,
                            "name": "INVERTED",
                            "description": "Inverted"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 6,
                "start": 4,
                "name": f"SDCLKout{p+1}_FMT",
                "description": "Sets the output format of the SYSREF clocks",
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "POWERDOWN",
                            "description": "Powerdown"
                        },
                        {
                            "value": 1,
                            "name": "LVDS",
                            "description": "LVDS"
                        },
                        {
                            "value": 2,
                            "name": "HSDS_6_MA",
                            "description": "HSDS 6 mA"
                        },
                        {
                            "value": 3,
                            "name": "HSDS_8_MA",
                            "description": "HSDS 8 mA"
                        },
                        {
                            "value": 4,
                            "name": "HSDS_10_MA",
                            "description": "HSDS 10 mA"
                        },
                        {
                            "value": 5,
                            "name": "LVPECL_1_6_V",
                            "description": "LVPECL 1600 mV"
                        },
                        {
                            "value": 6,
                            "name": "LVPECL_2_0_V",
                            "description": "LVPECL 2000 mV"
                        },
                        {
                            "value": 7,
                            "name": "LCPECL",
                            "description": "LCPECL"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 3,
                "start": 3,
                "name": f"DCLKout{p}_POL",
                "description": "Sets the polarity of the device clocks from the DCLKoutX outputs",
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "NORMAL",
                            "description": "Normal"
                        },
                        {
                            "value": 1,
                            "name": "INVERTED",
                            "description": "Inverted"
                        }
                    ]
                }
            },
            {
                "fieldtype": "normal",
                "end": 2,
                "start": 0,
                "name": f"DCLKout{p}_FMT",
                "description": "Sets the output format of the device clocks.",
                "valid": {
                    "type": "enum",
                    "values": [
                        {
                            "value": 0,
                            "name": "POWERDOWN",
                            "description": "Powerdown"
                        },
                        {
                            "value": 1,
                            "name": "LVDS",
                            "description": "LVDS"
                        },
                        {
                            "value": 2,
                            "name": "HSDS_6_MA",
                            "description": "HSDS 6 mA"
                        },
                        {
                            "value": 3,
                            "name": "HSDS_8_MA",
                            "description": "HSDS 8 mA"
                        },
                        {
                            "value": 4,
                            "name": "HSDS_10_MA",
                            "description": "HSDS 10 mA"
                        },
                        {
                            "value": 5,
                            "name": "LVPECL_1_6_V",
                            "description": "LVPECL 1600 mV"
                        },
                        {
                            "value": 6,
                            "name": "LVPECL_2_0_V",
                            "description": "LVPECL 2000 mV"
                        },
                        {
                            "value": 7,
                            "name": "LCPECL",
                            "description": "LCPECL"
                        }
                    ]
                }
            }
        ]
    }

In [10]:
with open("lmk04828b_regmap.json", "r") as f:
    regmap_data = json.load(f)

In [11]:
def transform_regmap(regmap_data, transformer, addresses, data):
    regmap_data = regmap_data.copy()
    for i in range(len(regmap_data)):
        addr = regmap_data[i]["addr"]
        if addr in addresses:
            di = addresses.index(addr)
            # print("Found addr, transforming:", hex(addr))
            regmap_data[i] = transformer(addr, data[di])
    
    return regmap_data

In [12]:
new_data = transform_regmap(regmap_data, get_clkout_odl_idl_div, [0x100, 0x108, 0x110, 0x118, 0x120, 0x128, 0x130], [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9), (10, 11), (12, 13)])
new_data = transform_regmap(new_data, get_clkout_ddly, [0x101, 0x109, 0x111, 0x119, 0x121, 0x129, 0x131], [0, 2, 4, 6, 8, 10, 12])
new_data = transform_regmap(new_data, get_clkout_ddlyd, [0x102, 0x10a, 0x112, 0x11a, 0x122, 0x12a, 0x132], [0, 2, 4, 6, 8, 10, 12])
new_data = transform_regmap(new_data, get_clkout_adly, [0x103, 0x10B, 0x113, 0x11B, 0x123, 0x12B, 0x133], [0, 2, 4, 6, 8, 10, 12])
new_data = transform_regmap(new_data, get_clkout_hs_mux, [0x104, 0x10C, 0x114, 0x11C, 0x124, 0x12C, 0x134], [0, 2, 4, 6, 8, 10, 12])
new_data = transform_regmap(new_data, get_sdclk_adly_en, [0x105, 0x10D, 0x115, 0x11D, 0x125, 0x12D, 0x135], [0, 2, 4, 6, 8, 10, 12])
new_data = transform_regmap(new_data, get_dclkout_ddly_pd, [0x106, 0x10E, 0x116, 0x11E, 0x126, 0x12E, 0x136], [0, 2, 4, 6, 8, 10, 12])
new_data = transform_regmap(new_data, get_sdclkout_pol_fmt, [0x107, 0x10F, 0x117, 0x11F, 0x127, 0x12F, 0x137], [0, 2, 4, 6, 8, 10, 12])

In [14]:
if 0:
    with open("lmk04828b_regmap_out.json", "w") as f:
        json.dump(new_data, f, indent=4)

# LMX2594 RF PLL

In [72]:
with open("lmx2594.txt", "r") as f:
    regmap_lines = f.read().strip().split("\n")

In [73]:
reg_pattern = re.compile(r"([a-zA-Z][a-zA-Z0-9_]*)(\[(\d+):(\d+)\])?(:(\d+))?")

In [74]:
regmap = []
registers = []

for li,line in enumerate(regmap_lines):
    words = line.split(" ")
    addr = int(words[0][1:])
    
    width = 0
    const_word = 0
    
    addr_bits = sum([(1 << i) * int(v) for i,v in enumerate(words[1:9][::-1])])
    # print(addr, addr_bits)
    
    reg = Register(addr)
    registers.append(reg)
    
    for i,word in enumerate(words[9:]):
        match = reg_pattern.match(word)

        if match is None:
            const_word |= int(word) << (15-width)
            width += 1
            continue
        
        regname,regpattern,end,start,_,w = match.groups()
        if regpattern is not None:
            regname += regpattern
        
        if end is not None:
            assert int(end) - int(start) + 1 == int(w)

        end = 15-width
        if w is not None:
            width += int(w)
        else:
            width += 1
        
        start = 16-width
        reg.add_field(end, start, regname)
        
    reg.add_field(15, 0, const_word)
    # print(f"{addr:2d}, 0b{const_word:016b}")
    
    assert width == 16, f"Line width {width} != 16: {line}"

In [71]:
with open("lmx2594_regmap_out.json", "w") as f:
    json.dump(registers, f, cls=RegisterSerializer, indent=4)